In [14]:
import torch
from torch . nn . functional import one_hot
import torch_explain as te
from torch_explain . nn . functional import l1_loss
from torch_explain . logic . nn import psi
from torch_explain . logic . metrics import test_explanation , complexity
import torch.nn.utils.prune as prune
import deep_logic as dl


# train data
x_train = torch . tensor ([
[0 , 0] ,
[0 , 1] ,
[1 , 0] ,
[1 , 1] ,
], dtype = torch . float )
y_train = torch . tensor ([0 , 1, 1, 0] , dtype = torch . float ). unsqueeze (1)

# instantiate a " psi network "
layers = [
    torch . nn . Linear ( x_train . shape [1] , 10) ,
    torch . nn . Sigmoid () ,
    torch . nn . Linear (10 , 5) ,
    torch . nn . Sigmoid () ,
    torch . nn . Linear (5 , 1) ,
    torch . nn . Sigmoid () ,
]
model = torch . nn . Sequential (* layers )

# fit ( and prune ) the model
optimizer = torch . optim . AdamW ( model . parameters () , lr =0.01)
loss_form = torch . nn . BCELoss ()
model . train ()
need_pruning = True
for epoch in range (6001) :
    optimizer . zero_grad ()
    y_pred = model ( x_train )
    loss = loss_form ( y_pred , y_train ) + 0.000001 * l1_loss ( model )
    loss . backward ()
    optimizer . step ()

    if epoch % 1000 == 0 :
        print ( 'epoch {} , loss {}'.format ( epoch , loss . item () ) )

    
    if epoch > 500 and need_pruning:
        dl.utils.relu_nn.prune_features(model, 2)
        need_pruning = False
    # model = prune_equal_fanin ( model , epoch , prune_epoch =1000 , k =2)

epoch 0 , loss 0.6935662031173706
Pruned 0/2 features
Pruned 0/2 features
epoch 1000 , loss 0.001324061886407435
epoch 2000 , loss 0.0005623179604299366
epoch 3000 , loss 0.00033241495839320123
epoch 4000 , loss 0.00021917466074228287
epoch 5000 , loss 0.0001542589598102495
epoch 6000 , loss 0.0001140840322477743


In [13]:
# get first - order logic explanations for a specific target class
y1h = one_hot ( y_train . squeeze () . long () )
print(y1h)

y_pred = model(y1h.float())

print(y_pred)

explanation = psi . explain_class ( model , x_train )

# compute explanation accuracy and complexity
accuracy , preds = test_explanation ( explanation , x_train , y1h , target_class =1)
explanation_complexity = complexity ( explanation )

tensor([[1, 0],
        [0, 1],
        [0, 1],
        [1, 0]])
tensor([[0.9998],
        [0.9998],
        [0.9998],
        [0.9998]], grad_fn=<SigmoidBackward0>)


ValueError: could not broadcast input array from shape (10,) into shape (2,)